<a href="https://colab.research.google.com/github/sksos7/Project/blob/main/Codestates_AI_05/Project_S_02/AI_05_%EC%9D%B4%EB%8F%99%EA%B7%9C_Section2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Second